In [1]:
# ИУ5-65Б СОРОКИН АРТЕМ




from sklearn.datasets import load_iris 
import pandas as pd
import numpy as np
import random

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV



iris = load_iris(return_X_y=False, as_frame=False)

In [2]:
print(type(iris))

<class 'sklearn.utils._bunch.Bunch'>


In [3]:
# data.head(10)

data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [4]:
print(data)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0       0.0  

In [ ]:
# СОЗДАНИE ПРОПУСКОВ


# ЧИСЛОВЫХ ПРИЗНАКОВ

for i in range(8):
    k = random.randint(1, 150)
    data['petal length (cm)'][k] = np.nan
    
# КАТЕГОРИАЛЬНЫЙ ПРИЗНАК    
for i in range(8):
    k = random.randint(1, 150)
    data['target'][k] = np.nan
    
    

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(data)

In [ ]:
### ПРОПУСКИ ДАННЫХ

In [ ]:
# Скопируем DataFrame чтобы попробовать несколько вариантов заполнения пропусков
# В данном случае, если проанализировать пропуски, становится понятно, что удобнее всего способ замены на значение 
# из следующей записи или из предыдущей. Так как записи в наборе данных более-менее отсортированные



# Попробуем сначала заполнить средним значением столбца (категориальный признак так заполнять не разумно)


df = data.copy()

df_test = df.fillna(df.mean())['petal length (cm)']

print (df.isnull().sum())

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_test)
    
    
# Среднее значение 3.733099. Выпадает из списка и может нарушить обучение модели в строках 27, 104, 122

In [ ]:
### Заполнение пропусков значением из предыдущей записи
### Крайне удачный вариант, так как категориальные признаки в данном случае отсортированы

In [ ]:
df_test = df.fillna(method='bfill')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_test)
    
# Получили практически исходный набор данных

### Перейдем к обучению модели

In [ ]:
iris.data.shape 

In [ ]:
x = iris.data
y = iris.target

### Делим данные на тестовую выборку и тренировочную

In [ ]:
#Делим данные на тренировочное и тестовое множество:

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, Y_test.shape, Y_train.shape)

### Будем использовать метод ближайших соседей, так как из данных видно, что классы имеют четко различимые границы

In [ ]:
K_range = range(1, 11)
scores = {}
scores_list = []
for k in K_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    scores[k] = metrics.accuracy_score(Y_test, Y_pred)
    scores_list.append(metrics.accuracy_score(Y_test, Y_pred))
    
print (scores)

### Посмотрим какой подберет гиперпараметр встроенный метод KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x, y)

In [ ]:
classes = {0:'setosa', 1:'versicolor', 2:'virginica'}

In [ ]:
X_new = [[3,4,5,2],
        [5,4,2,2,]]

In [ ]:
Y_pred = knn.predict(X_new)

In [ ]:
print (classes [Y_pred[0]])
print (classes [Y_pred[1]])

In [ ]:
#Создаем классификатор:

best_model = KNeighborsClassifier(
   n_neighbors=10, 
   weights='distance',
   algorithm='auto',
   leaf_size=30,
   metric='euclidean',
   metric_params=None,
   n_jobs=4
)

best_model.fit(X_train, Y_train)
predicted = best_model.predict(X_test)

print('Evaluation:\n', metrics.classification_report(Y_test, predicted))

### Видим, что модель может классифицировать данные, причем как показывает со 100% точностью. Дело в том, что числовые признаки сильно отличаются друг от друга при смене класса - это видно из набора данных. В таких случаях метод K-ближайших соседей показывает наибольшую точность прогнозирования.